In [23]:
import datetime
import gzip
import os
import shutil
from urllib.request import urlretrieve

import xarray as xr

In [5]:
"""
Get the filename of the .gz file. The same file name is used on 
the iastate.edu website and locally.
"""
def get_gz_fname(dt: datetime.date):
    dt_str = dt.strftime("%Y%m%d-%H%M%S")
    return f"MultiSensor_QPE_01H_Pass2_00.00_{dt_str}.grib2.gz"

In [6]:
def get_url(dt: datetime.date):
    date_str = dt.strftime("%Y/%m/%d")
    fname = get_gz_fname(dt)
    return (
        f"https://mtarchive.geol.iastate.edu/"
        f"{date_str}/mrms/ncep/MultiSensor_QPE_01H_Pass2/"
        f"{fname}"
    )


In [31]:
def get_hourly_data(dt: datetime):
    
    # define some path constants
    dir = os.path.abspath(os.path.join(os.path.curdir, "data"))
    if not os.path.isdir(dir):
        os.makedirs(dir)

    gz_fname = os.path.join(dir, get_gz_fname(dt))
    grib_fname = gz_fname.replace(".grib2.gz", ".grib2")
    nc_fname = gz_fname.replace(".grib2.gz", ".nc")

    # download the data
    url = get_url(dt)
    urlretrieve(url, gz_fname)

    # unzip
    with gzip.open(gz_fname, 'rb') as f_in:
        with open(grib_fname, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    
    # read into xarray
    ds = xr.open_dataset(grib_fname, engine="cfgrib").rename({"unknown": "precip_rate"})

    # cleanup
    os.remove(grib_fname)
    os.remove(gz_fname)

    return ds


In [32]:
dt = datetime.datetime(2021, 7, 5, 1)
ds = get_hourly_data(dt)
ds

<xarray.Dataset>
Dimensions:         (latitude: 3500, longitude: 7000)
Coordinates:
    time            datetime64[ns] ...
    step            timedelta64[ns] ...
    heightAboveSea  float64 ...
  * latitude        (latitude) float64 54.99 54.98 54.98 ... 20.03 20.02 20.01
  * longitude       (longitude) float64 230.0 230.0 230.0 ... 300.0 300.0 300.0
    valid_time      datetime64[ns] ...
Data variables:
    precip_rate     (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             161
    GRIB_centreDescription:  161
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             161
    history:                 2022-05-17T19:56 GRIB to CDM+CF via cfgrib-0.9.1...